In [ ]:
#Imports
import io

import tensorflow as tf
import pathlib
import matplotlib.pyplot as plt
import datetime

In [ ]:
datasetPath = '../Resources/KittiDataset/data_road/training/'
trainingImagesPath = pathlib.Path(datasetPath+'bev_image_2/')
trainingGTPath = pathlib.Path(datasetPath+'bev_gt_image_2/')
trainingLidarPath = pathlib.Path(datasetPath+'bev_velodyne/')

modelName = "ModelE"
time = datetime.datetime.now().strftime("%d_%m-%H_%M")
modelName = time+"-"+modelName

#For tensorboard log
base_log_dir = "logs/" + modelName
file_writer = None
tensorboard_callback = None



BUFFER_SIZE = 1000
BATCH_SIZE = 1
LEARNING_RATE = 0.0001
DROPOUT_RATE = 0.1

img_height = 800
img_width = 400
channels = 3


numClasses = 1
seed=1234
AUTOTUNE = tf.data.AUTOTUNE

AUGMENT = False
AUGMENT_RATE = 1

#If true, only 20/10 images will be used for training/validation
USE_SMALL_DATASET = True

In [ ]:
#Print a plt plot for Input, Ground Truth, Predicted mask (Predicted mask may not be neccesary)
def display_sample(display_list, title=None):
    fig = plt.figure(figsize=(5, 5))
    if title is not None:
        fig.suptitle(title, fontsize=16)
    title = ['BEV Image', 'True Mask', 'Predicted Mask']
    imagesList = [display_list[0], display_list[1], display_list[2]]
    if len(display_list) == 4:
        imagesList.append(display_list[3])
    for i in range(len(imagesList)):
        plt.subplot(1, len(imagesList), i + 1)
        plt.title(title[i])
        #plt.imshow(tf.keras.preprocessing.image.array_to_img(display_list[i]))
        plt.imshow(imagesList[i])
        plt.axis('off')
    plt.show()

#for sample_image,sample_mask in TrainingDataset.take(1):
#    display_sample([sample_image[0][0], sample_image[1][0], sample_mask[0]])

In [ ]:
def parse_img(img_path: str) -> dict:
  image = tf.io.read_file(img_path)
  image = tf.io.decode_png(image, channels=channels)
  image = tf.image.convert_image_dtype(image, tf.uint8)

  imageFileName = tf.strings.split(img_path, "/")[-1]
  splitedFileName = tf.strings.split(imageFileName, "_")

  imageType = splitedFileName[0]
  imageNumber = tf.strings.split(splitedFileName[1], ".")[0]

  maskFileName = imageType + "_road_" + imageNumber + ".png"

  mask_path = tf.strings.regex_replace(img_path, "bev_image_2", "bev_gt_image_2")
  mask_path = tf.strings.regex_replace(mask_path, imageFileName, maskFileName)

  maskImage = tf.io.read_file(mask_path)
  maskImage = tf.io.decode_png(maskImage, channels=channels)

  splittedMaskImage = tf.split(maskImage, 3, axis=2)
  maskImage = splittedMaskImage[2]

  image = tf.image.resize(image, (img_height, img_width))
  image = tf.cast(image, tf.float32) / 255.0

  maskImage = tf.image.resize(maskImage, (img_height, img_width)) /255
  maskImage = tf.cast(maskImage, tf.int8)



  return (image, maskImage)

def augment(data):
    img, mask = data

    img = img[0]
    mask = mask[0]

    #Flip left/right
    img = tf.image.flip_left_right(img)
    mask = tf.image.flip_left_right(mask)

    img = img[tf.newaxis, ...]
    mask = mask[tf.newaxis, ...]

    return img, mask

# Create a wrapper function for updating seeds.
def augmentingFunction(x, y):
    image, label = augment((x, y))
    return image, label

def augmentDataset(inputDataset):
    #print('Training dataset size before augmentation: ' + str(inputDataset.cardinality().numpy()))
    AugmentedDataset = inputDataset.take(round(inputDataset.cardinality().numpy() * AUGMENT_RATE))
    AugmentedDataset = AugmentedDataset.map(augmentingFunction)

    inputDataset = inputDataset.concatenate(AugmentedDataset)
    #print('Training dataset size after augmentation: ' + str(inputDataset.cardinality().numpy()))

    return inputDataset

def prepareDatasetKFolds(nFolds=10):
    FullDatasetSize = len(list(trainingImagesPath.glob('*.png')))
    foldSize = int(FullDatasetSize / nFolds)

    FullDataset =  tf.data.Dataset.list_files(str(trainingImagesPath/'*.png'), shuffle=False)
    FullDataset = FullDataset.map(parse_img)
    FullDataset = FullDataset.shuffle(buffer_size=BUFFER_SIZE, seed=seed, reshuffle_each_iteration=False)

    folds = []
    trainingSplits = []
    validationSplits = []

    for i in range(nFolds-1):
        folds.append(FullDataset.skip(foldSize*i).take(foldSize))

    skipped = foldSize*(nFolds-1)
    folds.append(FullDataset.skip(foldSize*(nFolds-1)))

    for i in range(nFolds):
        trainingFolds = folds[:i]
        trainingFolds.extend(folds[(i+1):])

        trainingSplit = FullDataset.take(0)
        for fold in trainingFolds:
            trainingSplit = trainingSplit.concatenate(fold)

        validationSplit = folds[i]

        trainingSplit = trainingSplit.batch(BATCH_SIZE)
        trainingSplit = trainingSplit.prefetch(buffer_size=AUTOTUNE)

        validationSplit = validationSplit.batch(BATCH_SIZE)
        validationSplit = validationSplit.prefetch(buffer_size=AUTOTUNE)

        trainingSplit = augmentDataset(trainingSplit)

        trainingSplits.append(trainingSplit)
        validationSplits.append(validationSplit)


    return trainingSplits, validationSplits

In [ ]:
def prepareModel():
    #Encoder1
    input1 = tf.keras.Input(shape=(img_height,img_width,channels), name='inputLayer_1')

    e1_c1_1 = tf.keras.layers.Conv2D(filters=16, kernel_size=(3,3), padding='same', activation='relu', name="e1_conv1_1")(input1)
    e1_c1_2 = tf.keras.layers.Conv2D(filters=16, kernel_size=(3,3), padding='same', activation='relu', name="e1_conv1_2")(e1_c1_1)
    e1_pool1 = tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=(2,2), padding='same', name="e1_pool1")(e1_c1_2)

    e1_c2_1 = tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), padding='same', activation='relu', name="e1_conv2_1")(e1_pool1)
    e1_c2_2 = tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), padding='same', activation='relu', name="e1_conv2_2")(e1_c2_1)
    e1_pool2 = tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=(2,2), padding='same', name="e1_pool2")(e1_c2_2)

    e1_c3_1 = tf.keras.layers.Conv2D(filters=64, kernel_size=(3,3), padding='same', activation='relu', name="e1_conv3_1")(e1_pool2)
    e1_c3_2 = tf.keras.layers.Conv2D(filters=64, kernel_size=(3,3), padding='same', activation='relu', name="e1_conv3_2")(e1_c3_1)
    e1_c3_3 = tf.keras.layers.Conv2D(filters=64, kernel_size=(3,3), padding='same', activation='relu', name="e1_conv3_3")(e1_c3_2)
    e1_pool3 = tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=(2,2), padding='same', name="e1_pool3")(e1_c3_3)

    e1_c4_1 = tf.keras.layers.Conv2D(filters=128, kernel_size=(3,3), padding='same', activation='relu', name="e1_conv4_1")(e1_pool3)
    e1_c4_2 = tf.keras.layers.Conv2D(filters=128, kernel_size=(3,3), padding='same', activation='relu', name="e1_conv4_2")(e1_c4_1)
    e1_c4_3 = tf.keras.layers.Conv2D(filters=128, kernel_size=(3,3), padding='same', activation='relu', name="e1_conv4_3")(e1_c4_2)
    e1_pool4 = tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=(2,2), padding='same', name="e1_pool4")(e1_c4_3)

    e1 = tf.keras.models.Model(inputs=[input1], outputs=e1_pool4, name="Encoder1")

    fc6 = tf.keras.layers.Dense(units=1024, name='fc6')(e1.output)
    drop6 = tf.keras.layers.Dropout(rate=DROPOUT_RATE, name='drop6')(fc6)

    fc7 = tf.keras.layers.Dense(units=1024, name='fc7')(drop6)
    drop7 = tf.keras.layers.Dropout(rate=DROPOUT_RATE, name='e2_drop7')(fc7)

    dc10 = tf.keras.layers.Conv2DTranspose(filters=128, kernel_size=16, strides=(2,2), padding='same', name='dconv10')(drop7)

    agX = tf.keras.layers.Add(name="agX")([dc10, e1_c4_3])

    c10_1 = tf.keras.layers.Conv2D(filters=128, kernel_size=(3,3), padding='same', activation='relu', name="conv10_1")(agX)
    c10_2 = tf.keras.layers.Conv2D(filters=128, kernel_size=(3,3), padding='same', activation='relu', name="conv10_2")(c10_1)
    c10_3 = tf.keras.layers.Conv2D(filters=128, kernel_size=(3,3), padding='same', activation='relu', name="conv10_3")(c10_2)

    dc11 = tf.keras.layers.Conv2DTranspose(filters=64, kernel_size=16, strides=(2,2), padding='same', name='dconv11')(c10_3)

    agY = tf.keras.layers.Add(name="agY")([dc11, e1_c3_3])

    c11_1 = tf.keras.layers.Conv2D(filters=64, kernel_size=(3,3), padding='same', activation='relu', name="conv11_1")(agY)
    c11_2 = tf.keras.layers.Conv2D(filters=64, kernel_size=(3,3), padding='same', activation='relu', name="conv11_2")(c11_1)
    c11_3 = tf.keras.layers.Conv2D(filters=64, kernel_size=(3,3), padding='same', activation='relu', name="conv11_3")(c11_2)

    dc12 = tf.keras.layers.Conv2DTranspose(filters=32, kernel_size=16, strides=(2,2), padding='same', name='dconv12')(c11_3)

    agZ = tf.keras.layers.Add(name="agZ")([dc12, e1_c2_2])

    c12_1 = tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), padding='same', activation='relu', name="conv12_1")(agZ)
    c12_2 = tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), padding='same', activation='relu', name="conv12_2")(c12_1)

    dc13 = tf.keras.layers.Conv2DTranspose(filters=16, kernel_size=16, strides=(2,2), padding='same', name='dconv13')(c12_2)
    c13_1 = tf.keras.layers.Conv2D(filters=16, kernel_size=(3,3), padding='same', activation='relu', name="conv13_1")(dc13)
    c13_2 = tf.keras.layers.Conv2D(filters=16, kernel_size=(3,3), padding='same', activation='relu', name="conv13_2")(c13_1)

    outputLayer = tf.keras.layers.Conv2D(filters=numClasses, kernel_size=16, strides=(1,1), padding='same', activation='sigmoid', name="output_layer")(c13_2)

    model = tf.keras.models.Model(inputs=[e1.inputs], outputs=[outputLayer])

    return model


In [ ]:
model = prepareModel()
model.summary()

In [ ]:
sample_image, sample_mask = None, None

def imageToTensorboard(epoch):
    global sample_image
    global sample_mask
    img = sample_image[0]
    mask = sample_mask[0]

    #Predict mask
    #one_img_batch = sample_image
    pred_mask = model.predict(sample_image)[0]
    display_list = [img, mask, pred_mask]

    #Create plot
    fig = plt.figure(figsize=(15, 5))
    title = ['Camera', 'True Mask', 'Predicted Mask']
    for i in range(len(display_list)):
        plt.subplot(1, len(display_list), i+1)
        plt.title(title[i])
        plt.imshow(display_list[i])
        plt.axis('off')

    #Convert plot to png image
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    plt.close(fig)
    buf.seek(0)
    img = tf.image.decode_png(buf.getvalue(), channels=4)
    img = tf.expand_dims(img, 0)

    with file_writer.as_default():
        tf.summary.image("Training", img, step=epoch+1)

class DisplayCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        imageToTensorboard(epoch=epoch)

In [ ]:
def train(TrainingDataset, ValidationDataset, foldNumber):
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE), loss=tf.keras.losses.BinaryCrossentropy(), metrics=['mse', tf.keras.metrics.BinaryIoU(target_class_ids=[1], name='BinaryIoU')], run_eagerly=False)

    stepsPerEpoch = TrainingDataset.cardinality().numpy() // BATCH_SIZE
    validationSteps = ValidationDataset.cardinality().numpy() // BATCH_SIZE

    global sample_data
    sample_data = ValidationDataset.take(1)


    callbacks = [
                 tensorboard_callback,
                 DisplayCallback(),
                 tf.keras.callbacks.ModelCheckpoint("./Models/"+modelName+"_Fold-"+str(foldNumber)+".h5", verbose=1, save_best_only=True),
                 tf.keras.callbacks.EarlyStopping(patience=10, verbose=1)
                 ]

    model_history = model.fit(TrainingDataset,
                          epochs=50,
                          validation_steps=validationSteps,
                          validation_data=ValidationDataset,
                          callbacks=callbacks)

    return model_history

In [ ]:
K = 10

trainingSplits, validationSplits = prepareDatasetKFolds(K)
for i, m in trainingSplits[0].take(1):
    sample_image = i
    sample_mask = m

model_history = None

val_binaryIoU_list = []

for trainSplit, validationSplit, i in zip(trainingSplits, validationSplits, list(range(K))):
    global model
    global file_writer
    global tensorboard_callback

    model = prepareModel()
    log_dir = base_log_dir + "/Fold_"+str(i)
    file_writer = tf.summary.create_file_writer(log_dir)
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

    print("Starting fold: " + str(i))
    model_history = train(trainSplit, validationSplit, i)
    val_binaryIoU = model_history.history['val_BinaryIoU'][-1]
    val_binaryIoU_list.append(val_binaryIoU)
    print("Ending fold: " + str(i))
    print("Validation BinaryIoU for fold " + str(i) + ": " + str(val_binaryIoU))
    print("#############################################################################3")

mean_val_binaryIoU = sum(val_binaryIoU_list) / len(val_binaryIoU_list)

with tf.summary.create_file_writer(base_log_dir).as_default():
    tf.summary.scalar('Mean_val_BinaryIoU', mean_val_binaryIoU, step=1)

print("Mean val BinaryIoU: "+str(mean_val_binaryIoU))

In [ ]:
print("Done")